In [2]:
import os

In [3]:
os.chdir("../")

In [4]:
%pwd

'/home/fmlpc/Shashank/Course_Work/MLOPS/MLOPS_Project_Skin_Cancer_Detection'

In [5]:
from dataclasses import dataclass
from pathlib import Path


@dataclass(frozen=True)
class DataIngestionConfig:
    root_dir: Path
    source_URL: str
    local_data_file: Path
    unzip_dir: Path

In [8]:
from Skin_Cancer_Classifier.constants import *
from Skin_Cancer_Classifier.utils.common import read_yaml, create_directories

In [9]:
class ConfigurationManager:
    def __init__(
        self,
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH):

        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)

        create_directories([self.config.artifacts_root])


    
    def get_data_ingestion_config(self) -> DataIngestionConfig:
        config = self.config.data_ingestion

        create_directories([config.root_dir])

        data_ingestion_config = DataIngestionConfig(
            root_dir=config.root_dir,
            source_URL=config.source_URL,
            local_data_file=config.local_data_file,
            unzip_dir=config.unzip_dir 
        )

        return data_ingestion_config

In [10]:
import os
import zipfile
import gdown
from Skin_Cancer_Classifier import logger
from Skin_Cancer_Classifier.utils.common import get_size


In [11]:
url = 'https://drive.google.com/file/d/15uNMnR3IoZaQYPoIAHJJeEfgACM5RBH3/view?usp=drive_link'
url.split("/")[-2]

'15uNMnR3IoZaQYPoIAHJJeEfgACM5RBH3'

In [12]:
class DataIngestion:
    def __init__(self, config: DataIngestionConfig):
        self.config = config

    
    def download_file(self)-> str:
        '''
        Fetch data from the url
        '''

        try: 
            dataset_url = self.config.source_URL
            zip_download_dir = self.config.local_data_file
            os.makedirs("artifacts/data_ingestion", exist_ok=True)
            logger.info(f"Downloading data from {dataset_url} into file {zip_download_dir}")

            file_id = dataset_url.split("/")[-2]
            prefix = 'https://drive.google.com/uc?id='
            gdown.download(prefix+file_id,zip_download_dir)

            logger.info(f"Downloaded data from {dataset_url} into file {zip_download_dir}")

        except Exception as e:
            raise e
        
    

    def extract_zip_file(self):
        """
        zip_file_path: str
        Extracts the zip file into the data directory
        Function returns None
        """
        unzip_path = self.config.unzip_dir
        os.makedirs(unzip_path, exist_ok=True)
        with zipfile.ZipFile(self.config.local_data_file, 'r') as zip_ref:
            zip_ref.extractall(unzip_path)


In [13]:
try:
    config = ConfigurationManager()
    data_ingestion_config = config.get_data_ingestion_config()
    data_ingestion = DataIngestion(config=data_ingestion_config)
    data_ingestion.download_file()
    data_ingestion.extract_zip_file()
except Exception as e:
    raise e

[2025-04-07 04:24:35,606: INFO: common: yaml file: config/config.yaml loaded successfully]
[2025-04-07 04:24:35,607: INFO: common: yaml file: params.yaml loaded successfully]
[2025-04-07 04:24:35,607: INFO: common: created directory at: artifacts]
[2025-04-07 04:24:35,608: INFO: common: created directory at: artifacts/data_ingestion]
[2025-04-07 04:24:35,608: INFO: 886113331: Downloading data from https://drive.google.com/file/d/15uNMnR3IoZaQYPoIAHJJeEfgACM5RBH3/view?usp=sharing into file artifacts/data_ingestion/data.zip]


Downloading...
From (original): https://drive.google.com/uc?id=15uNMnR3IoZaQYPoIAHJJeEfgACM5RBH3
From (redirected): https://drive.google.com/uc?id=15uNMnR3IoZaQYPoIAHJJeEfgACM5RBH3&confirm=t&uuid=f67127d9-2885-4ef4-932f-e2526a384b5b
To: /home/fmlpc/Shashank/Course_Work/MLOPS/MLOPS_Project_Skin_Cancer_Detection/artifacts/data_ingestion/data.zip
100%|██████████| 149M/149M [00:02<00:00, 55.4MB/s] 

[2025-04-07 04:24:43,382: INFO: 886113331: Downloaded data from https://drive.google.com/file/d/15uNMnR3IoZaQYPoIAHJJeEfgACM5RBH3/view?usp=sharing into file artifacts/data_ingestion/data.zip]
